# Importing the required libraries

In [1]:
pwd

'/home/wsuser/work'

In [2]:
import pandas as pd
import numpy as np 
import tensorflow #open source used for both ML and DL for computation

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

from keras.utils.np_utils import to_categorical

from keras.datasets import mnist

# Load data

In [3]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [4]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [6]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [7]:
mean = np.mean(X_train)
std = np.std(X_train)

def standardize(x):
    return (x-mean)/std

# Add CNN Layers

In [8]:
model=Sequential()

model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    

model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
    
model.add(Flatten())
model.add(Dense(512,activation="relu"))
    
model.add(Dense(10,activation="softmax"))
    
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 12, 12, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         1

In [10]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=15,
        zoom_range = 0.01,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)


train_gen = datagen.flow(X_train, Y_train, batch_size=128)
test_gen = datagen.flow(X_test, Y_test, batch_size=128)

# Train the model

In [11]:
epochs = 5
batch_size = 128
train_steps = X_train.shape[0] // batch_size
valid_steps = X_test.shape[0] // batch_size

es = keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10,
        verbose=1,
        mode="max",
        restore_best_weights=True,
     )

rp = keras.callbacks.ReduceLROnPlateau(
        monitor="val_accuracy",
        factor=0.2,
        patience=3,
        verbose=1,
        mode="max",
        min_lr=0.00001,
     )

In [12]:
history = model.fit(train_gen, 
                    epochs = epochs, 
                    steps_per_epoch = train_steps,
                    validation_data = test_gen,
                    validation_steps = valid_steps, 
                    callbacks=[es, rp])

Epoch 1/5
468/468 [==============================] - 354s 755ms/step - loss: 0.1287 - accuracy: 0.9590 - val_loss: 0.2205 - val_accuracy: 0.9296 - lr: 0.0010
Epoch 2/5
468/468 [==============================] - 352s 751ms/step - loss: 0.0539 - accuracy: 0.9836 - val_loss: 0.0622 - val_accuracy: 0.9800 - lr: 0.0010
Epoch 3/5
468/468 [==============================] - 356s 762ms/step - loss: 0.0438 - accuracy: 0.9865 - val_loss: 0.0450 - val_accuracy: 0.9855 - lr: 0.0010
Epoch 4/5
468/468 [==============================] - 348s 744ms/step - loss: 0.0380 - accuracy: 0.9883 - val_loss: 0.0463 - val_accuracy: 0.9851 - lr: 0.0010
Epoch 5/5
468/468 [==============================] - 355s 760ms/step - loss: 0.0347 - accuracy: 0.9889 - val_loss: 0.0408 - val_accuracy: 0.9890 - lr: 0.0010


# Save the model

In [14]:
model.save("mnistCNN.h5")

In [15]:
!tar -zcvf model.tgz mnistCNN.h5

mnistCNN.h5


In [19]:
ls

mnistCNN.h5  model.tgz


In [20]:
!pip install -U ibm-watson-machine-learning

# Cloud deploy

In [22]:
from ibm_watson_machine_learning import APIClient
credentials ={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"4e82RqZt6_F_pPgdXgqPdw30m5MFzTcjB3sfybHSgUyi"
}
client = APIClient(credentials)
client

In [23]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/620f87bf49fe4e3c92e766b48c5bd411:11798526-5b3f-4dab-be5c-f6ee2b0e612b::',
      'guid': '11798526-5b3f-4dab-be5c-f6ee2b0e612b',
      'name': 'Watson Machine Learning-xp',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'digit_recoginition',
    'scope': {'bss_account_id': '620f87bf49fe4e3c92e766b48c5bd411'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '0642be57-2093-4b1e-85c0-daaa20571327',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': '74c3082e75034f4fa23403c2fe2cd25d',
        'api_key': 'LSKehPCS4vRT4yO3dAl6mJVRbn1IP67xdYTyl10bUb1Q',
        'secret_access_key': 'a0e827c022218dc5eb92cf28acb68470bc52db2625301d32',
        'service_id': 'ServiceId-0167a5df-78c2-41ce-adbf-8b0e9b529201'},
       'editor': {'access_key_id': 'eda37defea5b4be59a9ed5b0d8ef538e',
        'a

In [24]:
def guid_from_space_name(client,deploy):
  space = client.spaces.get_details()
  return (next(item for item in space['resources'] if item['entity']['name']==deploy)['metadata']['id'])

In [26]:
space_uid = guid_from_space_name(client,'digit_recoginition')
print("Space UID = " + space_uid)

Space UID = a3d96776-f19d-4d9b-8b17-dc9f1b202663


In [27]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [29]:
software_space_uid = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [31]:
model_details = client.repository.store_model(model='model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN Digit recognition model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [32]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-19T11:50:17.357Z',
  'id': '033c20d9-3ff3-4874-bce2-bc893241b5ad',
  'modified_at': '2022-11-19T11:50:20.545Z',
  'name': 'CNN Digit recognition model',
  'owner': 'IBMid-665002OP6R',
  'resource_key': '5ed90903-081a-4b33-b573-65b4e82c8cb0',
  'space_id': 'a3d96776-f19d-4d9b-8b17-dc9f1b202663'},
 'system': {'warnings': []}}

In [33]:
model_id = client.repository.get_model_id(model_details)
model_id

'033c20d9-3ff3-4874-bce2-bc893241b5ad'

In [34]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

# Test Model

In [35]:
from keras.models import load_model
from keras.preprocessing import image
from PIL import Image
import numpy as np

In [36]:
model = load_model("mnistCNN.h5")

In [43]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='TKJO9c2OpsO4P_yWETKGcHVXWPutoh_1Aqf__oIl368E',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'digitrecoginition-donotdelete-pr-wqfpp3dbe9dpyc'
object_key = '3.jpg'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [45]:
img = Image.open(streaming_body_2).convert("L") 
img = img.resize( (28,28) )

In [46]:
img

In [47]:
im2arr = np.array(img)
im2arr = im2arr.reshape(1, 28, 28, 1) 

In [48]:
pred = model.predict(im2arr)
print(pred)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [49]:
print(np.argmax(pred, axis=1)) 

[3]
